In [9]:
import json
import pandas as pd

In [10]:
# Putting the JSON results file in a pandas DataFrame for easier calculations

# Opening the JSON file
with open('results.json', 'r') as f:
    data = json.load(f)

# Extract the movements
movements = data["Predictions"]

# Names of the columns
columns = ["T_x", "T_y", "T_z", "T_roll", "T_pitch", "T_yaw", "T_lin", "T_ang", 
           "P_x", "P_y", "P_z", "P_roll", "P_pitch", "P_yaw", "P_lin", "P_ang", 
           "D_x", "D_y", "D_z", "D_roll", "D_pitch", "D_yaw", "D_lin", "D_ang"]

# Creating the DataFrame
rows = []
for movement in movements:
    row = movement["Ground Truth"] + movement["Prediction"] + movement["Difference"]
    rows.append(row)

df = pd.DataFrame(rows, columns=columns)
df = df.apply(pd.to_numeric)

## 1. Components of movements that are the best / worst predicted

In [21]:
# Components of the movements
components = ["x", "y", "z", "roll", "pitch", "yaw", "lin", "ang"]

# Creating a DataFrame to show the results in a clear and synthetic way
df_results = pd.DataFrame({
    "Component": components,
    "Sum of Squared Differences": [(df[f"D_{c}"].abs()).sum()  for c in components],
    "Max absolute error": [df[f"D_{c}"].abs().max() for c in components],
    "Relative error": [round((df[f"D_{c}"] / df[f"T_{c}"]).abs()[df[f"T_{c}"] != 0].mean()*100,2) for c in components],
})

# Printing the DataFrame
print(df_results)

  Component  Sum of Squared Differences  Max absolute error  Relative error
0         x                       19.66                0.48           39.73
1         y                       25.76                0.73           35.28
2         z                        5.93                0.11             NaN
3      roll                        5.85                0.09             NaN
4     pitch                        4.52                0.12             NaN
5       yaw                       44.64                2.57           39.45
6       lin                       20.77                0.33            8.88
7       ang                       63.28                3.78          665.38


We don't take into account z, roll and pitch because the ground truth is always 0. But the fact that we still have some errors, shows that our model didn't remember thoses parameters but tried to estimate them instead. It's an indicator that goes against an overfitted model.

We can see that our model struggles most with forecasting the angular velocity. 

## 2. Best and worst predicted movements

In [20]:
# Adding, for each movements, the sum of the absolute errors
df["Total_Error"] = df[[f"D_{c}" for c in components]].abs().sum(axis=1)

# Keeping only the 5% best predicted movements (with the smaller total error)
best_predicted = df.nsmallest(int(len(df) * 0.05), "Total_Error")
print(best_predicted[[col for col in df.columns if col.startswith("T_")]])




      T_x   T_y  T_z  T_roll  T_pitch  T_yaw  T_lin  T_ang
208 -0.69  0.16  0.0     0.0      0.0   0.96   1.08   0.01
202 -0.47 -0.02  0.0     0.0      0.0   0.76   1.08  -0.21
207 -0.65  0.14  0.0     0.0      0.0   0.95   1.08   0.05
209 -0.73  0.17  0.0     0.0      0.0   0.97   1.08  -0.01
190 -1.33 -0.36  0.0     0.0      0.0   0.18   1.08  -0.02
88   2.11 -0.81  0.0     0.0      0.0  -0.30   1.08  -0.02
90   2.03 -0.84  0.0     0.0      0.0  -0.30   1.08  -0.02
191 -1.37 -0.36  0.0     0.0      0.0   0.18   1.08  -0.02
91   1.99 -0.85  0.0     0.0      0.0  -0.30   1.08  -0.02
11   0.03 -0.82  0.0     0.0      0.0   0.44   1.08  -0.04
79   0.51 -0.28  0.0     0.0      0.0  -0.37   0.56  -0.02
86   2.19 -0.79  0.0     0.0      0.0  -0.29   1.08  -0.08





In [19]:
# Keeping only the 5% worst predicted movements (with the higer total error)
worst_predicted = df.nlargest(int(len(df) * 0.05), "Total_Error")
print(worst_predicted[[col for col in df.columns if col.startswith("T_")]])




      T_x   T_y  T_z  T_roll  T_pitch  T_yaw  T_lin  T_ang
27  -0.04 -1.95  0.0     0.0      0.0   0.42   1.08  -3.54
61  -0.04 -0.37  0.0     0.0      0.0  -1.54   0.56   4.14
165 -0.30 -0.33  0.0     0.0      0.0  -0.04   1.08   1.41
28  -0.06 -2.02  0.0     0.0      0.0   0.09   1.08  -1.66
4   -0.02 -0.36  0.0     0.0      0.0   1.03   1.08  -3.39
192 -1.41 -0.36  0.0     0.0      0.0  -2.40   1.08  -0.02
168 -0.42 -0.30  0.0     0.0      0.0  -2.38   1.08   0.15
31  -0.18 -2.06  0.0     0.0      0.0   2.41   1.08  -0.19
179 -0.87 -0.38  0.0     0.0      0.0   0.18   1.08  -0.06
160 -0.13 -0.53  0.0     0.0      0.0   0.10   1.08  -0.98
193 -1.45 -0.36  0.0     0.0      0.0  -2.40   1.08  -0.02
170 -0.50 -0.31  0.0     0.0      0.0  -2.35   1.08   0.02





Indeed, we can see that our model struggles most when forecasting movements with a higher angular velocity. This is consistent with what we found before.